# https://map.naver.com/
- selenium4 를 검색해서 보세요!

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#요소에서 텍스트나 속성을 제어하는 것은 힘들다 
from bs4 import BeautifulSoup

driver=webdriver.Chrome()
url="https://map.naver.com/"
driver.get(url)  
#요청이 완료되면 driver 안에 html 문서가 포함되어 있다.(driver==브라우저)
inputSearch=driver.find_element(By.CLASS_NAME,"input_search")
inputSearch.send_keys("홍대맛집")
inputSearch.send_keys(Keys.ENTER)
#홍대 맛집 정보 수집 1.맛집 이름
soup=BeautifulSoup(driver.page_source,"html.parser") # 브라우저가 출력하고 있는 html 소스
titles=soup.select(".TYaxT")
titles 
#iframe에 출력중이라 안됨


[]

# iframe 
- iframe 은 html 문서 안아서 다른 html 을 불러오는 또다른 창이다.
- 보통 같은 서버내의 다른 문서를 불러오는데 사용한다.
- 한 브러우저에 2개의 창이 있는 형상 
- 네이버 맵의 메인 html 은 지도이고 검색하는 창이 iframe으로 되어 있다. 
- selenium 은 iframe 으로 불러온 html 을 로딩할 수 있다.

In [2]:
import time
# time.sleep(5) 
# 원시적인 방법 : 로딩되는 시간을 어림작아서 계산 (오류를 발생할 가능성이 존재, 연산시간이 늘어남)
driver.implicitly_wait(10) #10이내에 로딩이 완료되면 다음코드 실행
#iframe이 로딩될때는 완전한 방법이 될 수 없다.

from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC

# () 인자를 튜플로 받아서!!
wait(driver,10).until(EC.presence_of_element_located((By.ID,"searchIframe")))


<selenium.webdriver.remote.webelement.WebElement (session="ab381d74af1fd205016bb70cdea51999", element="f.04414CE5AD03A443BFFC5186749D9DED.d.BA35D374AE687EFB241254FB2AA203B2.e.167")>

In [3]:

#ifame 이 로딩되지 않았는데 찾고 바꾸려고하면 오류가 발생
#네이버 맵은 검색하지 않으면 iframe이 존재하지 않는다.
searchIframe=driver.find_element(By.ID,"searchIframe") 
#브라우저 내의 iframe을 id or name 속성으로 찾는다.
driver.switch_to.frame(searchIframe) #기준으로 삼고 있던 창을 바꾸겠다. 

shopNodes=[] #음식점 노드 리스트 
time.sleep(3)
container=driver.find_element(By.ID,"_pcmap_list_scroll_container")
container.click()
container.send_keys(Keys.END)
    
time.sleep(3)

#검색 결과의 1페이지 내용 
soup=BeautifulSoup(driver.page_source,"html.parser") # 브라우저가 출력하고 있는 html 소스
shopNodes+=soup.select(".UEzoS.rTjJo")



In [4]:
nextPageBtn=driver.find_element(By.CSS_SELECTOR,"div.zRM9F>a:last-child")
# nextPageBtn.click() # 요소를 마우스로 누르는 것
# aria-disabled 속성값이 "false" 면 next 버튼이 활성화,"true" 면 비활성화
# *html 내부의 속성 값은 무조건 문자열이다.
page=1
while nextPageBtn.get_attribute("aria-disabled") == "false":
    nextPageBtn.click()
    #driver(브라우저에) 다음페이지 정보가 들어온다.
    #page+=1
    #print(f"{(page)}페이지 입니다.")
    #page 을 동적으로 하고 있습니다.
    time.sleep(3) #ajax 페이징 정보(html)를 불러와서 div에 로딩중 (iframe과 유사)
    soup=BeautifulSoup(driver.page_source,"html.parser")
    shopNodes+=soup.select(".UEzoS.rTjJo")

    #titleNodes+=soup.select(".TYaxT")
    #potinNodes+=soup.select(".orXYY")

titles=[]
points=[]
for shop in shopNodes :
    titles.append(shop.select_one(".TYaxT").text)
    pointNode=shop.select_one(".orXYY")
    if pointNode != None :
        points.append(pointNode.text.replace("별점",""))
    else :
        points.append("X")



In [5]:
import pandas

data={"상호명":titles,"별점":points}

result=pandas.DataFrame(data)
result

,상호명,별점
0,감투,X
1,교장맥주 홍대입구역점,X
2,스케줄합정,X
3,오밀 홍대,X
4,단막 홍대합정점,X
5,기린1인용회전훠궈,X
6,대관령메밀막국수자연샤브샤브,X
7,카페 플로라운지,X
8,카레시 합정본점,X
9,비볼리 합정,4.56
